In [1]:
import pandas as pd
import torch

In [2]:
train = pd.read_csv("train.csv")
len(train)

9324

In [3]:
train.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-train-000,nsmc-sampled,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요.",2.2,0.0
1,boostcamp-sts-v1-train-001,slack-rtt,앗 제가 접근권한이 없다고 뜹니다;;,"오, 액세스 권한이 없다고 합니다.",4.2,1.0
2,boostcamp-sts-v1-train-002,petition-sampled,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.,2.4,0.0
3,boostcamp-sts-v1-train-003,slack-sampled,입사후 처음 대면으로 만나 반가웠습니다.,화상으로만 보다가 리얼로 만나니 정말 반가웠습니다.,3.0,1.0
4,boostcamp-sts-v1-train-004,slack-sampled,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!,0.0,0.0


neg sample을 잘 설계한다는 것은 사람이 봐도 헷갈릴만한 pair를 구성한다는 것이다. <br/>
3점 기준으로 나누어 진다면, 2.8점, 2.5점 이런 데이터들이 hard negative라고 볼 수 있다.  <br/>
binary-label==0인 데이터 중에서 2.5~2.9점 대의 데이터를 이용한다고하면, 해당 문장의 positive pair는 어떻게해야하는가? <br/>
→ 그러면 negative pair를 기준으로 pos pair는 main sentence 문장을 복사하자

contrastive dataset은 main/pos/neg 문장만 있으면 됨

In [63]:
def contrastive_data(data_path):
    """
    binary-label==0(neg pair)인 문장을 기준으로 contrastive learning dataframe을 생성
    main sent의 pos pair는 main sent와 동일한 문장으로 사용
    """
    train = pd.read_csv(data_path)
    neg_pair = train[train['binary-label']==0]
    final_df = pd.DataFrame()
    
    for row in neg_pair.iterrows():
        
        new_df_row = pd.DataFrame(
            {
                "main_sentence" : [row[1]["sentence_1"]],
                "pos_sentence" : [row[1]["sentence_1"]],
                "neg_sentence" : [row[1]["sentence_2"]]
            }
        )
        final_df = pd.concat([final_df, new_df_row], ignore_index=True)

    return final_df

In [40]:
temp = contrastive_data()

In [41]:
temp

,main_sentence,pos_sentence,neg_sentence
0,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,스릴도있고 반전도 있고 여느 한국영화 쓰레기들하고는 차원이 다르네요~,"반전도 있고,사랑도 있고재미도있네요."
1,주택청약조건 변경해주세요.,주택청약조건 변경해주세요.,주택청약 무주택기준 변경해주세요.
2,뿌듯뿌듯 하네요!!,뿌듯뿌듯 하네요!!,꼬옥 실제로 한번 뵈어요 뿌뿌뿌~!~!
3,이렇게 귀여운 쥐들은 처음이네요.ㅎㅎㅎ,이렇게 귀여운 쥐들은 처음이네요.ㅎㅎㅎ,이렇게 지겨운 공포영화는 처음..
4,미세먼지 해결이 가장 시급한 문제입니다!,미세먼지 해결이 가장 시급한 문제입니다!,가장 시급한 것이 신생아실 관리입니다!!!
...,...,...,...
5687,어쩌다 보니 사람 들어간 사진은 못찍었는데 너무 맛있었고 재밌었습니다!!,어쩌다 보니 사람 들어간 사진은 못찍었는데 너무 맛있었고 재밌었습니다!!,너무 맛있어서 또 가고싶은집!!
5688,동물보호법을 강화해 주세요,동물보호법을 강화해 주세요,동물원법을 강화시켜 주세요
5689,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,교원능력개발평가에서 교원이 보호받을 수 있는 장치를 마련해야합니다,본인이 납부한 국민연금 금액을 기준으로 대출을 받을 수 있는 제도를 마련해 주세요
5690,국회의원들 월급좀 줄여주세요,국회의원들 월급좀 줄여주세요,공무원 봉급좀 줄이지좀 마세요


In [42]:
temp.to_csv('./contrastive_train.csv', index=False)

In [67]:
def contrastive_data(data_path):
    """
    + pos pair 기준으로 만든 것도 추가! 
    
    binary-label==0(neg pair)인 문장을 기준으로 contrastive learning dataframe을 생성
    main sent의 pos pair는 main sent와 동일한 문장으로 사용
    """
    train = pd.read_csv(data_path)
    final_df = pd.DataFrame()
    
    neg_pair = train[train['binary-label']==0].reset_index(drop=True)
    for idx, row in neg_pair.iterrows():
        
        new_df_row = pd.DataFrame(
            {
                "main_sentence" : [row["sentence_1"]],
                "pos_sentence" : [row["sentence_1"]],
                "neg_sentence" : [row["sentence_2"]]
            }
        )
        final_df = pd.concat([final_df, new_df_row], ignore_index=True)
        
    # pos-pair 기준
    pos_pair = train[train["binary-label"]==1].reset_index(drop=True)
    for idx, row in pos_pair.iterrows():
        neg_idx = (idx + 1) % len(pos_pair)
        new_df_row = pd.DataFrame(
            {
                "main_sentence" : [row["sentence_1"]],
                "pos_sentence" : [row["sentence_2"]],
                "neg_sentence" : [pos_pair.iloc[neg_idx]["sentence_1"]]
            }
        )
        final_df = pd.concat([final_df, new_df_row], ignore_index=True)
    
    return final_df.sample(frac=1).reset_index(drop=True)

In [57]:
temp_2 = contrastive_data()

In [58]:
temp_2.to_csv('./contrastive_train_2.csv', index=False)

-----------

In [64]:
temp_3 = contrastive_data("dev.csv")

In [65]:
temp_3

,main_sentence,pos_sentence,neg_sentence
0,액션은개뿔 총몇번쏘고 끝입니다,액션은개뿔 총몇번쏘고 끝입니다,액션은 흉내만 내고 그마저도 후반부에는 슬로우모션 처리
1,다음 밥스테이지가 기대됩니다~ ㅎ,다음 밥스테이지가 기대됩니다~ ㅎ,다음 후기도 기대됩니다~~
2,사진 간지.. ㅋㅋ 축하드립니다,사진 간지.. ㅋㅋ 축하드립니다,사진이 나왔어요..ㅋㅋㅋ 축하드립니다.
3,영화보면서 기분전환이 되었어요,영화보면서 기분전환이 되었어요,영화 보는 내내 뭔가 모를 편안함을 느꼈어요
4,건강하게 자라기를!!,건강하게 자라기를!!,얼른 건강 회복하기를요!!!
...,...,...,...
259,국민청원제를 유료로 바꾸어야 합니다,국민청원제를 유료로 바꾸어야 합니다,종교법 매우 강력하게 바꾸어야합니다 ~ !
260,아직 입사 1주차인 제게 많은 도움이 되는 이야기들 해주셨습니다!!,아직 입사 1주차인 제게 많은 도움이 되는 이야기들 해주셨습니다!!,많은 조언해주셔서 목표와 생각을 정리하는데도 큰 도움이되었습니다!!ㅎㅎ
261,"사회적 이슈를 다루고 있는 가슴 찡한 드라마네요,,,","사회적 이슈를 다루고 있는 가슴 찡한 드라마네요,,,",정말 가슴을 따뜻하게 한 좋은 드라마...
262,내용이 뭔 내용인지도 모르겠음,내용이 뭔 내용인지도 모르겠음,무슨의미로 만들었는지 모르겠음..


In [66]:
temp_3.to_csv('./contrastive_dev.csv', index=False)

In [68]:
temp_4 = contrastive_data("dev.csv")

In [69]:
temp_4

,main_sentence,pos_sentence,neg_sentence
0,한고은 씨의 매력을 볼수있었던 드라마..,한고은 씨의 매력을 볼수있었던 드라마..,한석규의 전성시대가 열린 영화....
1,아무래도 상관 없어요.,아무래도 상관 없어요.,그것은 정말 중요하지 않습니다.
2,오! 사이즈가 딱 맞나용??,오! 사이즈가 딱 맞나용??,오… 거기서… 프로포즈하신 걸까요!..?
3,개인적으로 편보다 내용적인 면에서 만족했다.,"일단, 개인적으로 편보다 낫다고 생각한다.",상현 오피스에서 진행하자
4,맞아요 딱 정로환 ㅋㅋ,그렇군요 정로환입니다 ㅋㅋ,조만간 또 뵙겠습니다~
...,...,...,...
545,시작만 좋았던 영화,시작만 좋았던 영화,"모든것이 좋았던, 영화"
546,내용이 뭔 내용인지도 모르겠음,내용이 뭔 내용인지도 모르겠음,무슨의미로 만들었는지 모르겠음..
547,보다는 개연성 있는 줄거리와 볼수록 매혹되는 남 주인공~!!!,보다는 개연성 있는 줄거리와 볼수록 매혹되는 남 주인공~!!!,"처음엔 정말 허접해 보이지만, 보면 볼수록 빠져드는 매력적인 영화~!!"
548,비록 화상이지만 만나봬서 너무 반가웠습니다 .,오랜만에 봬서 너무 반가웠습니다,남들과다르다는이유만으로 억울한 죽음을 당한 브랜든 티나의 명복을 빕니다


In [70]:
temp_4.to_csv('./contrastive_dev_2.csv', index=False)